##Information Networks and College Choice 

Question: How do student information networks affect their application choices? And how do changing application choices change college offers?  

#Outline
  - Create Students with ASVAB score (truncated normal between 0 and 1) and "Income typical"(=0) or "Achievement Typical" (=1) designations
  - Assign students to high schools  
  - Allow students to influence their friends 
  - Setup social network within high schools 
  - Setup decision rule for choosing schools to apply to 
  - Setup decision rule for schools to choose students
  - Setup decision rule for choosing schools to attend

#I. Space

A. Social Network (Small world Graph) within high schools

B. Bipartite Network between high schools and colleges 

#II. Actors

##A. High School Students

In this model, students transmit information about cost and chance of admission to schools

###Student Properties
* student_ability: The private (not publicly known) ability of a student, drawn from a truncated normal distribution 
* student_score: The public proxy for ability of a student, which colleges use to select students 
* ability_rank: The rank among entire cohort based on ability
* score_rank: The rank among entire cohort based on score
* app_strategy: Either "income typical" or "achievement typical" 
      - income typical: apply to 1 school
      - achievement typical: apply to 5 schools
* student_apply: list of schools student applies to 
* student_attend: school student ultimately attends
* student_age: steps since initialization.
    * Freshman: 0
    * Soph: 1
    * Junior: 2
    * Senior: 3

##B. Colleges 

In this model, colleges are the entity into which students wish to sort optimally. 

###College Properties

- college_rank: The "Yellow Pages" rank of the college. That is, colleges ranked as institutions
- seat_rank: The "White Pages" rank of the college. That is, the first seat at the first-ranked school is one, but the first seat at the 5th-ranked school might be number 2000. 
- admit_standard: Schools admit students based on the number of seats they have. Admit the top X number of students with seats


#III. Initial Conditions

##A. Students

* Students are distributed into high school networks at initialization
* Student are given an "ability" that is randomly initialized with a draw from a truncated normal distribution
* Students are given a "score" which is the imperfect public display of skill students have, calculated using "ability" score and random noise
* Students have a rank within the entire cohort on both ability and score
* Students will have either achievement-typical status or income-typical status, drawn from a binomial distribution with its p determined by characteristics of the student population in a high school
* Students have a number of schools they apply to, with "cost" and "fit" affecting whether apply to some schools and not others, and the number of schools they apply to is determined in part by whether they are achievement or income typical
* Students have a college budget, the value of which is influenced by their high school income characteristic

##B. High Schools
* Has an income-characteristic (high, middle, low) of its student population, which determines the distribution of students to schools by app_strategy 
    - low income schools have a high number of income-typical students to start
    - middle income schools have roughly half and half 
    - high income schools have a high number of achievement-typcal students to start
* Space is a small world random graph connecting students of a given school
    
##C. Colleges
* Have a rank
* Have a number of seats

## D. Space containing both colleges and high schools 
* Bipartite network, in which students move from high schools to college
* Different lengths of edges in part determine "fit" for student

#IV. Model Parameters

Based on the description above, we need the following model parameters:

* num_high_schools: The number of high schools
* num_colleges: The number of colleges
* num_seats: The total number of seats in colleges 
* num_students: Total number of students
* at_prob: probability of being achievement typical
* it_prob: probability of being income typical
* switch_prob: probability of switching from IT -> AT, based on interactions with AT 
* min_college_budget, max_college_budget: ability to afford college prices

In [10]:
%matplotlib inline

# Standard imports
import copy
import itertools

# Scientific computing imports
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas
import seaborn; seaborn.set()

# Import widget methods
from IPython.html.widgets import *

#Student Class

* constructor: class constructor, which "initializes" or "creates" the student when we call Student(). This is in the __init__ method.

* decide_apply: students decide which colleges to apply to using rules based on their strategy
* decide_attend: students decide which college to attend using rules based on their strategy

In [1]:
class Student(object):
    
    def __init__(self, school_id, cohort_id, student_id, student_ability=0, score_rank=0, app_strategy="it", SAT=800, 
                 college_budget=1000, age=0, birthday=0):
        """
        Constructor for Student class.  
          * student ability drawn from a truncated normal distributon **between 0 and 1**
          * student score is ability times 2000 plus or minus 200
          * ranks are established from all students in that cohort across high schools
          * app strategy is either Income Typical or Achievement Typical
          
        Note that we must "link" the Person to their "parent" Model object.
        """
        # Set model link and ID
        self.school_id = school_id
        self.cohort_id = cohort_id
        self.student_id = student_id
        
        # Set student parameters 
        self.student_ability = student_ability
        self.college_budget = 5000
        self.SAT = SAT 
        self.score_rank = score_rank
        self.app_strategy = app_strategy
        self.age = age 
        self.SAT = (self.student_ability*2000)+np.random.randint(-200,200)
        self.birthday = birthday
        
    def age(self): 
        """
        Make students get older
        """
        
    def apply_college(self):
        """
        Have students choose colleges to apply to and apply
        Should occur at student age=12, or Q1 of senior year
        """
        #for 
        #if self.age==3:
            #if 
    def attend(self):
        """
        Have students choose colleges to attend 
        Should occur at student age=15, or Q4 of senior year
        """

#College Class

In [2]:
class College(object): 
    """
    College class, which accepts, rejects students and then contains matriculaters 
    """
    def __init__(self, model, college_id='a', rank=0, num_seats=0, admit_standard=0): 
    
        #Set model link and ID
        self.model = model
        
        #Set HS parameter
        self.college_id=college_id
        self.rank = rank
        self.num_seats = num_seats
        self.admit_standard = admit_standard
        
    def decide_admit(self):
        """
        Decide if applicant will be admitted
        """
        
    def send_info(self): 
        """
        Send information to students
        """

#Model Class 

constructor: class constructor, which "initializes" or "creates" the model when we call Model(). This is in the __init__ method.

setup_space: method to create our space

setup_students: method to create our people

setup_hs_space: method to create our high schools

setup_college_space: method to create our colleges 

get_num_at: method to measure strategy of connected peers 

step_be_friends: method to let friends influence student during step process


----------------

step_move: method to step through agent moves

step_interact: method to step through agent interaction

In [4]:
class Model(object):
    """
    Model Class, which encapsulates the entire behavior in each run of the model
    """
    def __init__(self, num_schools, num_colleges, num_cohorts=1, prob_out=.1, min_sticker_price=10000, 
                 max_sticker_price=50000, min_net_price=0, max_net_price=50000, 
                 min_college_budget=0, max_college_budget=100000, at_prob_l=.2, at_prob_m=.5, 
                 at_prob_h=.8, switch_prob=.6):
        
        #Set Model parameters 
        self.num_schools = num_schools
        self.num_colleges = num_colleges
        self.num_cohorts = num_cohorts
        self.prob_out= prob_out
        self.min_sticker_price = min_sticker_price
        self.max_sticker_price = max_sticker_price
        self.min_net_price = min_net_price
        self.max_net_price = max_net_price
        self.min_college_budget = min_college_budget
        self.max_college_budget = max_college_budget
        self.at_prob_l = at_prob_l
        self.at_prob_m = at_prob_m
        self.at_prob_h = at_prob_h
        self.switch_prob = switch_prob 
        
        #Set empty dictionary for storing graph
        self.student_graphs = {}
        self.school_list = string.ascii_uppercase
        
        
        # Set our state variables
        self.students = [] 
        self.colleges = []
        self.pop_seat_rank = {}
        self.t = 0
        self.num_cohorts_alt = 1
        
        # Call our setup methods to initialize high school space, students 
        self.setup_hs_space()
        self.create_college_space()
        
        
    def setup_hs_space(self):
        """
        Set up small world networks within highschools
        """

        # Iterate
        for school_id in self.school_list:

            # Setup empty school dictionary key
            self.student_graphs[school_id] = {}

            # Iterate through each cohort
            for cohort_id in range(self.num_cohorts):
                # Create graph randomly
                num_students = np.random.randint(50, 200)
                num_friends = 2 + np.random.binomial(4, 0.5)
                g = nx.newman_watts_strogatz_graph(num_students, num_friends, self.prob_out)

                # Setup empty cohort dictionary key 
                self.student_graphs[school_id][cohort_id] = {}

                # Create students
                for node_id in g.nodes():
                    # Set a student class as an attribute
                    g.node[node_id] = Student(school_id, cohort_id, node_id, 
                                              student_ability=sci.truncnorm.rvs(0, 1, size=1), 
                                              app_strategy=np.random.binomial(1, .5, 1))

                    # Set school-cohort graph into dictionary
                    self.student_graphs[school_id][cohort_id] = g 
                    
    def create_college_space(self):
        """
        Method to create high schools
        """
        college_list = string.ascii_lowercase
        rank = 0
        for college in college_list:
            rank +=1
            self.colleges.append(College(model=self, college_id=college,
                                         rank=rank, num_seats=np.random.randint(50,200)))
            
    def get_num_at(self):
        """
        Method to count number of income-typical students and achievement-typical students
        """
        income_typical = 0
        achievement_typical = 0 
        for school_id in self.school_list:
            for cohort_id in range(self.num_cohorts):
                for student in self.student_graphs[school_id][cohort_id]:
                    if self.student_graphs[school_id][cohort_id].node[student].app_strategy==1:
                        achievement_typical +=1
                    else: 
                        income_typical +=1
        print income_typical
        print achievement_typical
                    
    def step_be_friends(self, school_id, cohort_id, student_id): 
        """
        Method to allow students to change application strategy with friends' influence 
        """
        #Set up some relevant parameters
        prob_switch_per_friend = .1
        switch_count = 0 
        #Loop through schools, cohorts and students
        for school_id in self.school_list:
            for cohort_id in range(self.num_cohorts):
                prob_stat_switch = 0
                for student_id in student_graphs[school_id][cohort_id]:
                    at_friend_count = 0
                    #Only concern ourselves with influence on students who are now income typical
                    if student_graphs[school_id][cohort_id].node[student_id].app_strategy==0: 
                        for friend in student_graphs[school_id][cohort_id].neighbors(student_id):
                            #Check each friend to see their app strategy
                            if student_graphs[school_id][cohort_id].node[friend].app_strategy==1:
                                #Add 1 to Achievement Typical count for each AT friend
                                at_friend_count +=1
                                #Generate a probability of switching to that increases for each AT friend
                                prob_stat_switch = at_friend_count*prob_switch_per_friend
                        #Run process to determine if student switches to AT 
                        student_graphs[school_id][cohort_id].node[student_id].app_strategy=np.random.binomial(1, prob_stat_switch , 1)
                        strat = student_graphs[school_id][cohort_id].node[student_id].app_strategy
                        if strat==1:
                            #If student switches, add 1 to the running switch count
                            switch_count+=1
        
    def step(self):
        """
        Model step function.
        """
        # Increment steps
        self.t +=1
        
        # Increment age
        for school_id in self.school_list:
            for cohort_id in range(self.num_cohorts):
                for student_id in self.student_graphs[school_id][cohort_id]:
                    self.student_graphs[school_id][cohort_id].node[student_id].age +=1
        
        for school_id in self.school_list:
            for cohort_id in range(self.num_cohorts_alt):
                for student_id in self.student_graphs[school_id][cohort_id]:
                    print cohort_id
                    new_cohort_id = cohort_id+1
                    print new_cohort_id
                    self.student_graphs[school_id][new_cohort_id]=self.student_graphs[school_id][cohort_id]
                    del self.student_graphs[school_id][cohort_id]
                    self.num_cohorts_alt +=1
                    new_cohort_id = cohort_id+2
                    print new_cohort_id


#Results I Hope to Present
* Match rate = % of students at schools that are a good fit for their ability
* Mismatch based on misinformation versus application choices of students
* The various effects of cost parameters and the balance of decision components (fit vs cost preferences) on mismatch

#Hypotheses 
The literature says that most mismatch is driven by student choices about where to apply. I imagine then, that the driving force for many students will be their information network at high school and knowledge about costs.
